In [2]:
import tensorflow as tf
import numpy as np
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras.models import load_model
from keras import regularizers, optimizers
from keras.initializers import glorot_normal, RandomNormal, Zeros

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)# .reshape((y_train.shape[0], 10,1))
y_test = np_utils.to_categorical(y_test,num_classes)# .reshape((y_test.shape[0], 10,1))


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [4]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [5]:
activations = [e_swish_2, "relu", swish, e_swish_1]

names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_2', 'relu', 'swish', 'e_swish_1']


In [6]:
def create_model(act, act_name):
    nn = {"act": act, "act_name": act_name}
    
    weight_decay = 1e-2
    s = 2
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=glorot_normal(), input_shape=x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    
    # Block 2
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 3
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 4
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    
    # First Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    
    # Block 5
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 6
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 7
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    # Second Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    
    # Block 8
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 9
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    # Third Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    
    # Block 10
    model.add(Conv2D(512, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    # Block 11  
    model.add(Conv2D(2048, (1,1), padding='same', kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 12  
    model.add(Conv2D(256, (1,1), padding='same', kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fourth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))


    # Block 13
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fifth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    # Final Classifier
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    nn["model"] = model
    return nn

In [7]:
models = []

In [ ]:
for i, act in enumerate(activations):
    batch_size = 128
    nn = create_model(act, names[i])
    
    nn["model"].summary()
    model = nn["model"]

    # First training for 50 epochs
    epochs = 25*2
    opt_adm = keras.optimizers.Adadelta()
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_1 = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_1"] = part_1.history
    print(nn["part_1"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    # Training for 30 epochs more
    epochs = 30
    opt_adm = keras.optimizers.Adadelta(lr=0.7, rho=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_2= model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_2"] = part_2.history
    print(nn["part_2"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    # First training for 25 epochs
    epochs = 25
    opt_adm = keras.optimizers.Adadelta(lr=0.6, rho=0.90)
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_3 = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_3"] = part_3.history
    print(nn["part_3"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    # First training for 50 epochs
    epochs = 20
    opt_adm = keras.optimizers.Adadelta(lr=0.5, rho=0.95)
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_4 = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_4"] = part_4.history
    print(nn["part_4"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    del nn["model"]
    print("\n", nn) # Ensure everything's ok x2
    models.append(nn)
     

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 128)       0         
__________

Epoch 1/30
391/390 [==============================] - 193s 493ms/step - loss: 0.1927 - acc: 0.9326 - val_loss: 0.2567 - val_acc: 0.9149
Epoch 2/30
391/390 [==============================] - 188s 482ms/step - loss: 0.1840 - acc: 0.9351 - val_loss: 0.2507 - val_acc: 0.9156
Epoch 3/30
391/390 [==============================] - 188s 482ms/step - loss: 0.1774 - acc: 0.9381 - val_loss: 0.2570 - val_acc: 0.9156
Epoch 4/30
391/390 [==============================] - 188s 482ms/step - loss: 0.1751 - acc: 0.9382 - val_loss: 0.2500 - val_acc: 0.9190
Epoch 5/30
391/390 [==============================] - 188s 482ms/step - loss: 0.1726 - acc: 0.9394 - val_loss: 0.2479 - val_acc: 0.9217
Epoch 6/30
391/390 [==============================] - 188s 482ms/step - loss: 0.1717 - acc: 0.9398 - val_loss: 0.2548 - val_acc: 0.9188
Epoch 7/30
391/390 [==============================] - 188s 482ms/step - loss: 0.1677 - acc: 0.9406 - val_loss: 0.2620 - val_acc: 0.9132
Epoch 8/30
391/390 [============================

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_11 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
activation_14 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 128)       512       
_________________________________________________________________
activation_15 (Activation)   (None, 32, 32, 128)       0         
__________

Epoch 1/30
391/390 [==============================] - 156s 400ms/step - loss: 0.1706 - acc: 0.9409 - val_loss: 0.2421 - val_acc: 0.9197
Epoch 2/30
391/390 [==============================] - 150s 385ms/step - loss: 0.1599 - acc: 0.9434 - val_loss: 0.2408 - val_acc: 0.9229
Epoch 3/30
391/390 [==============================] - 150s 385ms/step - loss: 0.1558 - acc: 0.9457 - val_loss: 0.2502 - val_acc: 0.9190
Epoch 4/30
391/390 [==============================] - 151s 385ms/step - loss: 0.1530 - acc: 0.9457 - val_loss: 0.2340 - val_acc: 0.9242
Epoch 5/30
391/390 [==============================] - 151s 385ms/step - loss: 0.1521 - acc: 0.9454 - val_loss: 0.2481 - val_acc: 0.9177
Epoch 6/30
391/390 [==============================] - 151s 385ms/step - loss: 0.1514 - acc: 0.9468 - val_loss: 0.2463 - val_acc: 0.9212
Epoch 7/30
391/390 [==============================] - 151s 385ms/step - loss: 0.1493 - acc: 0.9489 - val_loss: 0.2483 - val_acc: 0.9198
Epoch 8/30
391/390 [============================

In [8]:
for i, act in zip([2,3], activations[-2:]):
    batch_size = 128
    nn = create_model(act, names[i])
    
    nn["model"].summary()
    model = nn["model"]

    # First training for 50 epochs
    epochs = 25*2
    opt_adm = keras.optimizers.Adadelta()
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_1 = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_1"] = part_1.history
    print(nn["part_1"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    # Training for 30 epochs more
    epochs = 30
    opt_adm = keras.optimizers.Adadelta(lr=0.7, rho=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_2= model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_2"] = part_2.history
    print(nn["part_2"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    # First training for 25 epochs
    epochs = 25
    opt_adm = keras.optimizers.Adadelta(lr=0.6, rho=0.90)
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_3 = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_3"] = part_3.history
    print(nn["part_3"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    # First training for 50 epochs
    epochs = 20
    opt_adm = keras.optimizers.Adadelta(lr=0.5, rho=0.95)
    model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
    part_4 = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
    nn["part_4"] = part_4.history
    print(nn["part_4"])
    model.save_weights("simplenet_generic_"+nn["act_name"]+"_"+str(epochs)+".h5")

    del nn["model"]
    print("\n", nn) # Ensure everything's ok x2
    models.append(nn)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 128)       0         
__________

391/390 [==============================] - 160s 410ms/step - loss: 0.6597 - acc: 0.7696 - val_loss: 0.5688 - val_acc: 0.8058
Epoch 6/50
391/390 [==============================] - 160s 410ms/step - loss: 0.6042 - acc: 0.7904 - val_loss: 0.5182 - val_acc: 0.8245
Epoch 7/50
391/390 [==============================] - 160s 410ms/step - loss: 0.5649 - acc: 0.8050 - val_loss: 0.5157 - val_acc: 0.8249
Epoch 8/50
391/390 [==============================] - 160s 409ms/step - loss: 0.5258 - acc: 0.8155 - val_loss: 0.5515 - val_acc: 0.8138
Epoch 9/50
391/390 [==============================] - 160s 410ms/step - loss: 0.4970 - acc: 0.8281 - val_loss: 0.4384 - val_acc: 0.8509
Epoch 10/50
391/390 [==============================] - 160s 409ms/step - loss: 0.4697 - acc: 0.8379 - val_loss: 0.3877 - val_acc: 0.8694
Epoch 11/50
391/390 [==============================] - 160s 410ms/step - loss: 0.4474 - acc: 0.8457 - val_loss: 0.3947 - val_acc: 0.8660
Epoch 12/50
391/390 [==============================] - 16

Epoch 1/30
391/390 [==============================] - 165s 421ms/step - loss: 0.1402 - acc: 0.9512 - val_loss: 0.2259 - val_acc: 0.9258
Epoch 2/30
391/390 [==============================] - 160s 410ms/step - loss: 0.1316 - acc: 0.9533 - val_loss: 0.2298 - val_acc: 0.9282
Epoch 3/30
391/390 [==============================] - 160s 410ms/step - loss: 0.1281 - acc: 0.9554 - val_loss: 0.2190 - val_acc: 0.9314
Epoch 4/30
391/390 [==============================] - 160s 410ms/step - loss: 0.1258 - acc: 0.9553 - val_loss: 0.2262 - val_acc: 0.9279
Epoch 5/30
322/390 [=======================>......] - ETA: 26s - loss: 0.1224 - acc: 0.9569

KeyboardInterrupt: 